In [22]:
from ultralytics import YOLO

video_file = 'D:/Titulo/Github/vehicle_video_trajectory_extractor/videos/video1_30s.mp4'

# Load a pretrained YOLOv8n model
model = YOLO('D:/Titulo/Github/vehicle_video_trajectory_extractor/models/cutom_dota.pt')

# Run inference on 'bus.jpg' with arguments
results = model(video_file, conf=0.2, iou=0.5, classes=None, agnostic_nms=False, augment=False, verbose=False, save=True)
# model.predict('D:/Titulo/Github/vehicle_video_trajectory_extractor/videos/video1_30s.mp4', save=True, conf=0.2)





WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

Results saved to runs\detect\predict18


In [23]:
# function to extract specific frame array from video
import cv2

def GetFrame(frame_number, path = video_file):
    vidcap = cv2.VideoCapture(path)
    vidcap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    success, image = vidcap.read()
    return image

In [14]:
labels = list(results[0].names.values())

boxes = [frame.boxes.data.tolist() for frame in results]  # una lista de predicciones con los bbox de cada frame
# bbox structure = [[[x1, y1, x2, y2, conf, cls]... ] **n_frames**]



[436.59814453125,
 432.3736572265625,
 451.889892578125,
 443.0633544921875,
 0.8976417183876038,
 1.0]

In [24]:
from ikomia.dataprocess.workflow import Workflow
from ikomia.utils.displayIO import display
import cv2

# Initialize the workflow
wf = Workflow()

# Add DeepSORT tracking algorithm
tracking = wf.add_task(name="infer_deepsort", auto_connect=True)
tracking.set_parameters({"categories": "all", "conf_thres": "0.5"})

x=0
for frame_data in boxes:
    x+=1
    frame = GetFrame(x)
    wf.run_on(array=frame)

    # Get results
    image_out = tracking.get_output(0)
    obj_detect_out = tracking.get_output(1)

    # Convert the result to BGR color space for displaying
    img_out = image_out.get_image_with_graphics(obj_detect_out)
    img_res = cv2.cvtColor(img_out, cv2.COLOR_RGB2BGR)

    # Display or save the frame as needed
    display(img_res, title="DeepSORT", viewer="opencv")

# Clean up
cv2.destroyAllWindows()


Ikomia auto-completion updated for installed plugins.
Ikomia auto-completion updated for Ikomia HUB algorithms.
Try installing infer_deepsort from Ikomia HUB...


infer_deepsort.zip: 100%|██████████| 824k/824k [00:00<00:00, 938kiB/s] 

Installing infer_deepsort requirements. This may take a while, please be patient...


Command '['c:\\Users\\jflir\\AppData\\Local\\Programs\\Python\\Python38\\python.exe', '-m', 'pip', 'install', '-r', 'C:\\Users\\jflir\\Ikomia\\Plugins\\Python\\infer_deepsort\\requirements2.txt']' returned non-zero exit status 1.


RuntimeError: Algorithm infer_deepsort could not be loaded:<class 'ModuleNotFoundError'>: No module named 'torchreid':   File "C:\Users\jflir\Ikomia/Plugins/Python\infer_deepsort\infer_deepsort_process.py", line 23, in <module>
    from infer_deepsort.deep_sort_pytorch.deep_sort import DeepSort

  File "C:\Users\jflir\Ikomia/Plugins/Python\infer_deepsort\deep_sort_pytorch\deep_sort\__init__.py", line 1, in <module>
    from .deep_sort import DeepSort

  File "C:\Users\jflir\Ikomia/Plugins/Python\infer_deepsort\deep_sort_pytorch\deep_sort\deep_sort.py", line 4, in <module>
    from .deep.feature_extractor import Extractor

  File "C:\Users\jflir\Ikomia/Plugins/Python\infer_deepsort\deep_sort_pytorch\deep_sort\deep\feature_extractor.py", line 8, in <module>
    from torchreid import models
 (Code 9)

In [2]:
import numpy as np
import cv2
from ultralytics import YOLO
from sort import Sort

if __name__ == '__main__':
    cap = cv2.VideoCapture("D:/Titulo/Github/vehicle_video_trajectory_extractor/videos/video1_30s_sift_estabilizado_filtrado.mp4")

    model = YOLO("D:/Titulo/Github/vehicle_video_trajectory_extractor/models/cutom_dota.pt")

    tracker = Sort()

    while cap.isOpened():
        status, frame = cap.read()

        if not status:
            break

        results = model(frame, stream=True)

        for res in results:
            filtered_indices = np.where(res.boxes.conf.cpu().numpy() > 0.5)[0]
            boxes = res.boxes.xyxy.cpu().numpy()[filtered_indices].astype(int)
            tracks = tracker.update(boxes)
            tracks = tracks.astype(int)
            
            for xmin, ymin, xmax, ymax, track_id in tracks:
                cv2.putText(img=frame, text=f"Id: {track_id}", org=(xmin, ymin-10), fontFace=cv2.FONT_HERSHEY_PLAIN, fontScale=2, color=(0,255,0), thickness=2)
                cv2.rectangle(img=frame, pt1=(xmin, ymin), pt2=(xmax, ymax), color=(0, 255, 0), thickness=2)

        # frame = results[0].plot()

        cv2.imshow("frame", frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()


0: 576x1024 1 large vehicle, 96 small vehicles, 100.1ms
Speed: 10.0ms preprocess, 100.1ms inference, 17.0ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 1 large vehicle, 104 small vehicles, 50.0ms
Speed: 6.0ms preprocess, 50.0ms inference, 2.0ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 102 small vehicles, 42.0ms
Speed: 7.0ms preprocess, 42.0ms inference, 5.0ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 1 large vehicle, 102 small vehicles, 23.0ms
Speed: 6.0ms preprocess, 23.0ms inference, 4.0ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 1 large vehicle, 97 small vehicles, 8.0ms
Speed: 6.0ms preprocess, 8.0ms inference, 2.0ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 1 large vehicle, 98 small vehicles, 8.0ms
Speed: 7.0ms preprocess, 8.0ms inference, 3.0ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 1 large vehicle, 99 small vehicles, 8.0ms
Speed: 5.0ms preprocess, 8.0ms infe